# Reading Parquet Files using DuckDB

In this example, we will use Ibis's DuckDB backend to analyze data from a remote parquet source.
This process should work locally, as well as with CSV.

To do this, we will be looking at the [**Global Biodiversity Information Facility (GBIF) Species Occurrences**](https://registry.opendata.aws/gbif/) dataset.
It is hosted on S3 at `s3://gbif-open-data-us-east-1/occurrence/`

We will focus on observations posted in April 2023, so our full path should be `s3://gbif-open-data-us-east-1/occurrence/2023-04-01/occurrence.parquet/*`.

## Reading One Partition

We can read a single partition by specifying its path.

We do this by calling [`read_parquet`](https://ibis-project.org/api/expressions/top_level/#ibis.read_parquet) on the partition we care about.

So to read the first partition in this dataset, we'll call `read_parquet` on `00000` in that path:

In [ ]:
import ibis

t = ibis.read_parquet(f"s3://gbif-open-data-us-east-1/occurrence/2023-04-01/occurrence.parquet/000000")
t

Note that we're calling `read_parquet` and receiving a table expression without establishing a connection first.
Ibis spins up a DuckDB connection (or whichever default backend you have) when you call `ibis.read_parquet` (or even `ibis.read_csv`).

Since our result, `t`, is a table expression, we can now run queries against the file using Ibis expressions.
For example, we can select columns, filter the file, and then view the first five rows of the result:

In [ ]:
cols = ['gbifid', 'datasetkey', 'occurrenceid', 'kingdom',
        'phylum', 'class', 'order', 'family', 'genus',
        'species', 'day', 'month', 'year']

t.select(cols).filter(t['family'].isin(['Corvidae'])).limit(5).execute()

or count the rows in the table (partition):

In [ ]:
t.count().execute()

## Reading All Partitions: Filter, Aggregate, Export
We can use `read_parquet` to read an entire parquet file by globbing all partitions:

In [ ]:
t = ibis.read_parquet(f"s3://gbif-open-data-us-east-1/occurrence/2023-04-01/occurrence.parquet/*")

and since the function returns a table expression, we can perform valid selections, filters, aggregations, and exports just as we could with any other table expression:

In [ ]:
df = (
    t.select(['gbifid', 'family', 'species'])
    .filter(t['family'].isin(['Corvidae']))
    # Here we limit by 10,000 to fetch a quick batch of results
    .limit(10000)
    .group_by('species')
    .count()
    .execute()
)

print(df.shape)
df.head()